In [ ]:
# 查看环境安装情况
!pip freeze | grep tensor

In [ ]:
# 查看文件行数
!wc -l data/poetry/*.txt

In [ ]:
# 查看文件内容
!head -5 data/poetry/*.txt

In [ ]:
# 多文件操作
with open('data/poetry/raw.txt', 'r') as rawfp,\
  open('data/poetry/input.txt', 'w') as infp,\
  open('data/poetry/output.txt', 'w') as outfp:
    
    prev_line = ''
    for curr_line in rawfp:
        curr_line = curr_line.strip()
        # poems break at empty lines, so this ensures we train only
        # on lines of the same poem
        if len(prev_line) > 0 and len(curr_line) > 0:       
            infp.write(prev_line + '\n')
            outfp.write(curr_line + '\n')
        prev_line = curr_line

In [ ]:
#github忽略文件
touch .gitignore
vi .gitignore
# 忽略文件夹
/doc/

In [ ]:
# 合并checkpoints
python avg_checkpoints.py --checkpoints=[tacotron_model.ckpt-345000,tacotron_model.ckpt-350000] --output_path="/Users/liuxinxin/Downloads/new.ckpt"

In [ ]:
# tf.reduce_mean 说明
a = tf.random_uniform([2,3],minval=5,
    maxval=5,)
# [[5. 5. 5.]
#  [5. 5. 5.]]
b = tf.reduce_mean(a,-1)
# [5. 5.]
b = tf.reduce_mean(a)
# 5.0
b = tf.reduce_mean(a,-1,keepdims=True)
# [[5.]
#  [5.]]

In [ ]:
# 使用request访问api
import requests

sentences = "测试语句"
url = 'http://127.0.0.1:19877/?sentences=' + sentences
crawl_content = requests.get(url)
crawl_content.text

In [ ]:
# 异步访问
async def call_coroutine(session, url,i,files):
    with async_timeout.timeout(60):
        async with session.get(url) as response:
            content = await response.text()
            content = json.loads(content)
            files[i] = content['wav_file']
            print(content['wav_file'])
            print(i)
            return await response.release()
 
 
async def main(loop,sentences,files):
    async with aiohttp.ClientSession(loop=loop) as session:
        tasks = [call_coroutine(session, 'http://127.0.0.1:8099/?sentences=' + sentence,i,files) for i,sentence in enumerate(sentences)]
        await asyncio.gather(*tasks)


def synic(sentences):
    start = time()
    files = {}
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main(loop,sentences,files))
    files  = sorted(files.items(),key = lambda item:item[0])
    files = [file[1]+'.wav'  for file in files]
    if len(files) > 1:
        break_wav = ["480ms.wav"] * (len(files) - 1)
        j = 1
        for i, ele in enumerate(break_wav):
            files.insert(i + j, ele)
            j += 1
        files = ['wav_out/' + filename for filename in files]
        cbn = sox.Combiner()
        nowTime = datetime.now().strftime('%Y%m%d%H%M%S')
        output_file = 'wav-' + nowTime + '.wav'
        cbn.build(files, 'wav_out/' + output_file, 'concatenate')
    else:
        output_file = files[0]
    stop = time()
    print(stop - start)
    return output_file

In [ ]:
# 设置互信
# 生成秘钥
ssh-keygen -t rsa
# 把id_rsa.pub内容写入authorized_keys中
# 修改权限
chmod 700 .ssh #.ssh权限
chmod 600 authorized_keys

In [ ]:
# ubuntu 压缩文件
 tar -zcvf workspace.tar.gz worksapce

In [1]:
# 生成传输数据
server_admin = "zpf"
server_address = "106.120.3.69"
server_dir = "/home/zpf/exp/tacotron2/"
home_dir = "/Users/liuxinxin/Downloads/"
file_name = "inference.ipynb"

send_command = "scp -P 22 "+ home_dir + "/" + file_name + " " + server_admin + "@" + server_address + ":" + server_dir
rece_command = "scp -P 22 "+ server_admin + "@" + server_address + ":" + server_dir + file_name + " " + home_dir 
print("send command:")
print(send_command)
print("rece_command:")
print(rece_command)

send command:
scp -P 22 /Users/liuxinxin/Downloads//inference.ipynb zpf@106.120.3.69:/home/zpf/exp/tacotron2/
rece_command:
scp -P 22 zpf@106.120.3.69:/home/zpf/exp/tacotron2/inference.ipynb /Users/liuxinxin/Downloads/


In [ ]:
# 查看端口占用情况
lsof -i tcp:8099

In [ ]:
# json不能传输ndarray的问题，如果解决
import numpy as np
import json

# ndarray 无法序列化
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

a = np.array([1, 2, 3])
print(json.dumps({'aa': [2, (2, 3, 4), a], 'bb': [2]}, cls=NumpyEncoder))

In [ ]:
# mac sh 文件可以执行
chmod u+x

In [ ]:
# 大文件查询
du --max-depth=1 -h .

In [ ]:
# 安装redis
下载https://redis.io/download
$ cd redis-3.2.8
$ make
$ sudo make install 
# 运行
/usr/local/bin
redis-server

In [ ]:
# 使用redis
import redis   # 导入redis模块，通过python操作redis 也可以直接在redis主机的服务端操作缓存数据库

r = redis.Redis(host='localhost', port=6379, decode_responses=True)   # host是redis主机，需要redis服务端和客户端都启动 redis默认端口是6379
r.set('name', 'junxi')  # key是"foo" value是"bar" 将键值对存入redis缓存
print(r['name'])
print(r.get('name'))  # 取出键name对应的值
print(type(r.get('name')))
